In [1]:
## Function for getting house links from a webpage at willow

def get_url(soup_links):
    
    df_links = []
    links = soup_links.findAll('a', attrs={'class': "list-card-link list-card-link-top-margin list-card-img"})
    for link in links:
        result = link.get('href') # returns the text part of an entire document or a tag
        df_links.append(result)
    return df_links 


In [2]:
## Function 

In [3]:
## Extracting individual house links from willow for different cities.

import urllib.request 
import urllib.parse
import urllib.error
import ssl
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as BS
import pandas as pd
import time
import os
import re
import json
import datetime
import numpy as np

# For ignoring SSL certificate errors

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

#base url for the website
base_url ='https://www.zillow.com/{}-ca/1_p/?searchQueryState=%7B%22pagination%22%3A%7B%22currentPage%22%3A2%7D%2C%22mapBounds%22%3A%7B%22west%22%3A-122.04668998718262%2C%22east%22%3A-121.92961692810059%2C%22south%22%3A37.53736331242%2C%22north%22%3A37.61232767789535%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A11540%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A13%7D'

cities=['San-Jose','San-Francisco','South-San-Francisco','San-Mateo','Palo-Alto','Santa-Clara','Cupertino','Milpitas','Fremont','Union-City','Hayward',
        'Castro-Valley','Berkeley','Oakland','Pleasanton','Dublin','San-Ramon','Danville','Burlingame']

links_array=[]

for city in cities:
    url=base_url.format(city)
#     print("creating link for city {}".format(url))
    
    # Making the website believe that you are accessing it using a mozilla browser
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    
    # Creating a BeautifulSoup object of the html page for easy extraction of data.
    soup = BS(webpage,'html.parser')
    # calling function to get the links
    links_array.extend(get_url(soup)) 

# print(len(links_array)) 
print(links_array)# copied these 760 links in a file named links.txt in the working directory.


['https://www.zillow.com/homedetails/388-Henderson-Dr-San-Jose-CA-95123/19822172_zpid/', 'https://www.zillow.com/homedetails/1953-Ceylon-Ave-San-Jose-CA-95122/19729713_zpid/', 'https://www.zillow.com/homedetails/6185-Mahan-Dr-San-Jose-CA-95123/19823063_zpid/', 'https://www.zillow.com/homedetails/2102-Bello-Ave-San-Jose-CA-95125/19692875_zpid/', 'https://www.zillow.com/homedetails/2490-Fairglen-Dr-San-Jose-CA-95125/19692922_zpid/', 'https://www.zillow.com/homedetails/1228-Sandra-Dr-San-Jose-CA-95125/19685366_zpid/', 'https://www.zillow.com/homedetails/1479-Sierraville-Ave-San-Jose-CA-95132/19771768_zpid/', 'https://www.zillow.com/homedetails/1121-Norval-Way-San-Jose-CA-95125/19685582_zpid/', 'https://www.zillow.com/homedetails/1778-Coastland-Ave-San-Jose-CA-95125/19686450_zpid/', 'https://www.zillow.com/homedetails/377-Conestoga-Way-San-Jose-CA-95123/19822324_zpid/', 'https://www.zillow.com/homedetails/6237-Hancock-Ave-San-Jose-CA-95123/19833212_zpid/', 'https://www.zillow.com/homedetai

In [4]:
# writing the urls into files and opening them

def create_soup_file_from_link():
    '''
    You can create 760 soup object files, one for each house link, by calling this function.
   
   '''
    
#     with open('/Users/niveditasharma/Documents/Business_Analytics_CSUEB/BAN_612/Project/links.txt') as f:
    with open('links.txt') as f:    
        lines = f.readlines()

    for house_url in lines:
        house_url = house_url.strip('\n')
        req_house = Request(house_url, headers={'User-Agent': 'Mozilla/5.0'})
        time.sleep(1) #waiting time for downloading the html data properly from webpage
        web_page_house = urlopen(req_house).read()
        time.sleep(1)
        soup_house = BS(web_page_house, "html.parser")
        time.sleep(1)
        zpid = house_url.split('_zpid')[0].split('/')[-1]
        # Below is the code to create the soup object and write it to a file
        
#         with open('/Users/niveditasharma/Documents/Business_Analytics_CSUEB/BAN_612/Project/soup_data/soup_object_' + zpid, 'w', encoding='utf-8') as so:
        with open('soup_data/soup_object_' + zpid, 'w', encoding='utf-8') as so:
            so.write(str(soup_house))

        if soup_house.find('script', id="hdpApolloPreloadedData", type="application/json") is not None:
            print("Found the Data of {}".format(zpid))
        else:
            print("Data is None of {}".format(zpid))

In [5]:
## Creating 760 soup object files, one for each house link, by calling this function.

# create_soup_file_from_link()

In [6]:
## Getting data from Soup object files

def get_property_details():
    
#     dir_name = '/Users/niveditasharma/Documents/Business_Analytics_CSUEB/BAN_612/Project/soup_data/'
    dir_name="soup_data/" 
    property_details_list=[]

    for dirs in os.listdir(dir_name):
        try:
            zpid = dirs.split('_')[-1]
            with open(dir_name + dirs) as soup_file:
                soup_content = soup_file.read()


            soup = BS(soup_content, "html.parser") # to convert back the string to soup object

            script_tag = soup.find('script', id="hdpApolloPreloadedData", type="application/json")
            value = re.findall(r'<script id="hdpApolloPreloadedData" type="application/json">(.*?)</script>',
                               script_tag.decode())
            json_data = json.loads("".join(value))
            final_json = json.loads(json_data['apiCache'])
            
            if 'ForSaleDoubleScrollFullRenderQuery' in soup_content:
                tag_value = 'ForSaleDoubleScrollFullRenderQuery{"zpid":' + zpid + ',"contactFormRenderParameter":{"zpid":' + zpid + ',"platform":"desktop","isDoubleScroll":true}}'
            elif 'NewConstructionInitialRenderSEOQuery' in soup_content:
                tag_value = 'NewConstructionInitialRenderSEOQuery{"zpid":' + zpid + '}'
            elif 'OffMarketDoubleScrollFullRenderQuery' in soup_content:
                tag_value = 'OffMarketDoubleScrollFullRenderQuery{"zpid":' + zpid + ',"contactFormRenderParameter":{"zpid":' + zpid + ',"platform":"desktop","isDoubleScroll":true}}'
            else:
                tag_value = None

            if tag_value:

                property_details = final_json[tag_value]['property']
                property_details_list.append(property_details) 

            else:
                print("Tag Value is None")
                
        except Exception as err:
            print("Program Failed with {}".format(err))
            
    return property_details_list    


In [7]:
# saving the list of dictionaries in a variable
property_details_complete = get_property_details()

## Getting column datas for dataframe

In [8]:
def get_address_from_soup_files(details):
      
    addresslist=[]
    streetAdress=[]
    city=[]
    state=[]
    zipcode=[]
    neighborhood=[]
    subdivision=[] 
    
    for detail in details:
        address=detail.get('address',None)
        addresslist.append(address)
    
    for add in addresslist:
        if add:
            street_add=add.get('streetAddress',None)
            streetAdress.append(street_add)

            city_add=add.get('city',None)
            city.append(city_add)

            state_add=add.get('state',None)
            state.append(state_add)

            zipcode_add=add.get('zipcode',None)
            zipcode.append(zipcode_add)

            neighborhood_add=add.get('neighborhood',None)
            neighborhood.append(neighborhood_add)

            subdivision_add=add.get('subdivision', None)
            subdivision.append(subdivision_add)
       
        else:
            streetAdress.append('None')
            city.append('None')
            state.append('None')
            zipcode.append('None')
            neighborhood.append('None')
            subdivision.append('None')
            
        
    df_address=pd.DataFrame({'Street':streetAdress, 'City':city, 'State': state, 'Zipcode':zipcode,
                           'Neighbourhood': neighborhood,'Subdivision':subdivision})

    return df_address
          

In [9]:
def get_features_from_soup_files(details):
      
    bedrooms=[]
    bathrooms=[]
    price_list=[]
    zestimate_list=[]
    year_built_list=[]
    longitude_list=[]
    latitude_list=[]
    living_area_list=[]
    home_type_list=[]  
    lot_size_list=[]
    HOA_list=[]
    brokerage_name=[]
    last_sold_price_list=[]
    zpid_list=[]
    home_status_list=[]
    
    for detail in details:
        bedroom=detail.get('bedrooms',None)
        bedrooms.append(bedroom)

        bathroom=detail.get('bathrooms',None)
        bathrooms.append(bathroom)

        price=detail.get('price',None)
        price_list.append(price)
        
        zestimate=detail.get('zestimate',None)
        zestimate_list.append(zestimate)

        year_built=detail.get('yearBuilt',None)
        year_built_list.append(year_built)

        longitude=detail.get('longitude',None)
        longitude_list.append(longitude)

        latitude=detail.get('latitude',None)
        latitude_list.append(latitude)

        living_area=detail.get('livingArea',None)
        living_area_list.append(living_area)

        home_type=detail.get('homeType',None)
        home_type_list.append(home_type)

        lot_size=detail.get('lotSize',None)
        lot_size_list.append(lot_size)

        HOA=detail.get('monthlyHoaFee',None)
        HOA_list.append(HOA)

        brokerage=detail.get('brokerageName',None)
        brokerage_name.append(brokerage) 
        
        last_sold_price=detail.get('lastSoldPrice',None)
        last_sold_price_list.append(last_sold_price)
        
        zpid=detail.get('zpid',None)
        zpid_list.append(zpid)
        
        home_status=detail.get('homeStatus',None)
        home_status_list.append(home_status)
           

    df_features=pd.DataFrame({ 'Price':price_list,'Zestimate':zestimate_list,'Bedroom': bedrooms, 'Bathroom': bathrooms, 'Year_Built':year_built_list,
                                 'Longitude':longitude_list, 'Latitude':latitude_list, 'Living_Area(Sqft)':living_area_list,
                                 'Home_Type':home_type_list,'Lot_Size(Sqft)':lot_size_list,'HOA_Per_Month':HOA_list,
                                 'Brokerage':brokerage_name,'Last_Sold_Price':last_sold_price_list,'Zpid':zpid_list,'Home_Status':home_status_list})
    return df_features
        

In [24]:
def get_price_history_from_soup_files(details):
    
    price_history_list_tag=[]
    event_list=[]
    date_list=[]
    price_history_list=[]
    
    for detail in details:
        price_history_tag=detail.get('priceHistory',None)
        if price_history_tag:        
            price_history_list_tag.append(price_history_tag[0])
        else:    
            price_history_list_tag.append(None)

          
    for price in price_history_list_tag:
        if price:
            event=price.get('event',None)
            event_list.append(event)

            time=price.get('time',None)
            datetime_date = datetime.date.fromtimestamp(time//1000)
            date_list.append(datetime_date)

            price_history=price.get('price',None)
            price_history_list.append(price_history)
        else:
            event_list.append(None)
            date_list.append(None)
            price_history_list.append(None)
            
    df_price_history=pd.DataFrame({'Price_History':price_history_list, 'Event':event_list,'Date':date_list})

    return df_price_history

In [25]:
def get_agent_from_soup_files(details):
    
    listing_provider_list=[]
    agent_name_list=[]
    agent_phone_no_list=[]
    posting_group_name_list=[]
    
    
    for detail in details:
        listing_provider=detail.get('listingProvider',None)
        listing_provider_list.append(listing_provider)
    
    for provider in listing_provider_list:
        if provider:
            agent_name=provider.get('agentName',None)
            agent_name_list.append(agent_name)
            
            agent_phone_no=provider.get('phoneNumber',None)
            agent_phone_no_list.append(agent_phone_no)
            
            posting_group_name=provider.get('postingGroupName',None)
            posting_group_name_list.append(posting_group_name)
            
         
        else:
            agent_name_list.append(None)
            agent_phone_no_list.append(None)
            posting_group_name_list.append(None)
            
    df_agent_details=pd.DataFrame({'Agent_Name':agent_name_list,'Agent_Phone_No.':agent_phone_no_list,'Posting_Group_Name':posting_group_name_list})  
    return df_agent_details
    

In [26]:

def get_Nearby_Schools_details_from_soup_files(details):
    nearby_schools_list=[]
    school_name_list=[]
    school_rating_list=[]
    
    for detail in details:
        nearby_schools=detail.get('schools',None)
        nearby_schools_list.append(nearby_schools)
        
    for schools in nearby_schools_list:
        if schools: 
            if len(schools)==1:
                schools.extend([None,None])
            elif len(schools)==2:
                schools.append(None)            
            for school in schools: 
                if school and school != 'None':
                    school_name=school.get('name',None)
                    school_name_list.append(school_name)

                    school_rating=school.get('rating',None)
                    school_rating_list.append(school_rating)

                else:
            
                    school_name_list.append(None)
                    school_rating_list.append(None)
        else: 
            school_name_list.extend(['None','None','None'])
            school_rating_list.extend(['None','None','None'])

    np_array_name=np.array(school_name_list).reshape(760,3)
    np_array_rating=np.array(school_rating_list).reshape(760,3)
    df_nearby_schools_name=pd.DataFrame(np_array_name,columns=['First_Nearby_School_Name','Second_Nearby_School_Name','Third_Nearby_School_Name'])
    df_nearby_schools_rating=pd.DataFrame(np_array_rating,columns=['First_Nearby_School_Rating','Second_Nearby_School_Rating','Third_Nearby_School_Rating'])
    
    df_nearby_schools=df_nearby_schools_name.join(df_nearby_schools_rating)
     
    return df_nearby_schools.reindex(columns=['First_Nearby_School_Name','First_Nearby_School_Rating','Second_Nearby_School_Name','Second_Nearby_School_Rating','Third_Nearby_School_Name','Third_Nearby_School_Rating'])



In [27]:
def get_listing_details_from_soup_files(details):
    
    listing_sub_type_list=[]
    is_new_home_list=[]
    is_for_auction_list=[]
    is_bank_owned_list=[]
    
    for detail in details:
        listing_sub_type=detail.get('listing_sub_type',None)
        listing_sub_type_list.append(listing_sub_type)
        
    for listing in listing_sub_type_list:
        if listing:
            is_new_home=listing.get('is_newHome',None)
            is_new_home_list.append(is_new_home)
            
            is_for_auction=listing.get('is_forAuction',None)
            is_for_auction_list.append(is_for_auction)
            
            is_bank_owned=listing.get('is_bankOwned',None)
            is_bank_owned_list.append(is_bank_owned)
            
        else:
            is_new_home_list.append(None)
            is_for_auction_list.append(None)
            is_bank_owned_list.append(None)
     
    df_listing_details=pd.DataFrame({'Is_New_Home':is_new_home_list,'Is_For_Auction':is_for_auction_list,'Is_Bank_Owned':is_bank_owned_list})
    return df_listing_details

In [28]:
def get_reso_facts_from_soup_files(details):
    hasHeating_list=[]
    hasCooling_list=[]
    hasFireplace_list=[]
    laundryFeatures_list=[]
    furnished_list=[]
    buildingArea_list=[]
    parking_list=[]
    hasGarage_list=[]
    garageSpaces_list=[]
    hasOpenParking_list=[]
    hasCarport_list=[]
    levels_list=[]
    hasPrivatePool_list=[]
    hasSpa_list=[]
    hasView_list=[]
    frontageType_list=[]
    architecturalStyle_list=[]
    roofType_list=[]
    propertyCondition_list=[]
    isNewConstruction_list=[]
    developmentStatus_list=[]
    hasHomeWarranty_list=[]
    hasElectricOnProperty_list=[]
    greenBuildingVerificationType_list=[]
    hasPetsAllowed_list=[]
    hasRentControl_list=[]
    securityFeatures_list=[]

    
    for detail in details:
        reso_facts=detail.get('resoFacts',None)
        
        if reso_facts:
            has_heating=reso_facts.get('hasHeating',None)
            hasHeating_list.append(has_heating)
            
            hasCooling=reso_facts.get('hasCooling',None)
            hasCooling_list.append(hasCooling)
            
            laundryFeatures=reso_facts.get('laundryFeatures',None)
            if laundryFeatures:
                laundryFeatures_list.append(",".join(laundryFeatures))
            else:
                laundryFeatures_list.append(None)
            
            hasFireplace=reso_facts.get('hasFireplace',None)
            hasFireplace_list.append(hasFireplace)
            
            furnished=reso_facts.get('furnished',None)
            furnished_list.append(furnished)
            
            buildingArea=reso_facts.get('buildingArea',None)
            buildingArea_list.append(buildingArea)
            
            parking=reso_facts.get('parking',None)
            parking_list.append(parking)
            
            hasGarage=reso_facts.get('hasGarage',None)
            hasGarage_list.append(hasGarage)
            
            garageSpaces=reso_facts.get('garageSpaces',None)
            garageSpaces_list.append(garageSpaces)
            
            hasOpenParking=reso_facts.get('hasOpenParking',None)
            hasOpenParking_list.append(hasOpenParking)
            
            hasCarport=reso_facts.get('hasCarport',None)
            hasCarport_list.append(hasCarport)
            
            levels=reso_facts.get('levels',None)
            levels_list.append(levels)
            
            hasPrivatePool=reso_facts.get('hasPrivatePool',None)
            hasPrivatePool_list.append(hasPrivatePool)
            
            hasSpa=reso_facts.get('hasSpa',None)
            hasSpa_list.append(hasSpa)
            
            hasView=reso_facts.get('hasView',None)
            hasView_list.append(hasView)
            
            frontageType=reso_facts.get('frontageType',None)
            frontageType_list.append(frontageType)
            
            architecturalStyle=reso_facts.get('architecturalStyle',None)
            architecturalStyle_list.append(architecturalStyle)
            
            roofType=reso_facts.get('roofType',None)
            roofType_list.append(roofType)
            
            propertyCondition=reso_facts.get('propertyCondition',None)
            propertyCondition_list.append(propertyCondition)
            
            isNewConstruction=reso_facts.get('isNewConstruction',None)
            isNewConstruction_list.append(isNewConstruction)
            
            developmentStatus=reso_facts.get('developmentStatus',None)
            developmentStatus_list.append(developmentStatus)
            
            hasHomeWarranty=reso_facts.get('hasHomeWarranty',None)
            hasHomeWarranty_list.append(hasHomeWarranty)
            
            hasElectricOnProperty=reso_facts.get('hasElectricOnProperty',None)
            hasElectricOnProperty_list.append(hasElectricOnProperty)
            
            greenBuildingVerificationType=reso_facts.get('greenBuildingVerificationType',None)
            greenBuildingVerificationType_list.append(greenBuildingVerificationType)
            
            hasPetsAllowed=reso_facts.get('hasPetsAllowed',None)
            hasPetsAllowed_list.append(hasPetsAllowed)
            
            hasRentControl=reso_facts.get('hasRentControl',None)
            hasRentControl_list.append(hasRentControl)
            
            securityFeatures=reso_facts.get('securityFeatures',None)
            if securityFeatures:
                securityFeatures_list.append(",".join(securityFeatures))
            else:
                securityFeatures_list.append(None)
            
        else:
            hasHeating_list.append(None)
            hasCooling_list.append(None)
            hasFireplace_list.append(None)
            furnished_list.append(None)
            buildingArea_list.append(None)
            parking_list.append(None)
            hasGarage_list.append(None)
            garageSpaces_list.append(None)
            hasOpenParking_list.append(None)
            hasCarport_list.append(None)
            levels_list.append(None)
            hasPrivatePool_list.append(None)
            hasSpa_list.append(None)
            hasView_list.append(None)
            frontageType_list.append(None)
            architecturalStyle_list.append(None)
            roofType_list.append(None)
            propertyCondition_list.append(None)
            isNewConstruction_list.append(None)
            developmentStatus_list.append(None)
            hasHomeWarranty_list.append(None)
            hasElectricOnProperty_list.append(None)
            greenBuildingVerificationType_list.append(None)
            hasPetsAllowed_list.append(None)
            hasRentControl_list.append(None)
            securityFeatures_list.append(None)
            
    df_reso_facts=pd.DataFrame({'hasHeating':hasHeating_list, 'hasCooling':hasCooling_list, 'hasFireplace':hasFireplace_list,'laundryFeatures':laundryFeatures_list, 'furnished':furnished_list,'buildingArea':buildingArea_list,
                             'parking':parking_list,'hasGarage':hasGarage_list,'garageSpaces':garageSpaces_list,
                              'hasOpenParking':hasOpenParking_list, 'hasCarport':hasCarport_list, 'levels':levels_list,
                              'hasPrivatePool':hasPrivatePool_list, 'hasSpa':hasSpa_list, 'hasView':hasView_list,
                               'frontageType':frontageType_list,'architecturalStyle':architecturalStyle_list,
                               'roofType':roofType_list,'propertyCondition':propertyCondition_list,'isNewConstruction':isNewConstruction_list,
                               'developmentStatus':developmentStatus_list,'hasHomeWarranty':hasHomeWarranty_list,
                               'hasElectricOnProperty':hasElectricOnProperty_list,'greenBuildingVerificationType':greenBuildingVerificationType_list,
                               'hasPetsAllowed':hasPetsAllowed_list,'hasRentControl':hasRentControl_list,'securityFeatures':securityFeatures_list})       
    return  df_reso_facts      

In [29]:
# Main Function

if __name__ == "__main__":

    dataframe_address = get_address_from_soup_files(property_details_complete)
    # print(dataframe_address.head(10))
    dataframe_features = get_features_from_soup_files(property_details_complete)
#     print(dataframe_features.head(10))
    dataframe_price_history = get_price_history_from_soup_files(property_details_complete)
    # print(dataframe_price_history.head(10))
    dataframe_agent=get_agent_from_soup_files(property_details_complete)
    # print(dataframe_agent.head(10))
    dataframe_Nearby_Schools_details= get_Nearby_Schools_details_from_soup_files(property_details_complete)
    # print(dataframe_Nearby_Schools_details.head(10))
    dataframe_listing_details=get_listing_details_from_soup_files(property_details_complete)
    # print(dataframe_listing_details.head(10))
    dataframe_reso_facts=get_reso_facts_from_soup_files(property_details_complete)
    # print(dataframe_reso_facts)

    df_realestate = pd.concat([dataframe_address,dataframe_features,dataframe_reso_facts,dataframe_listing_details,dataframe_price_history,dataframe_agent,dataframe_Nearby_Schools_details], axis=1)
#     print( df_realestate.head(20))
    print(df_realestate.shape[0])



760


In [16]:
df_realestate.to_csv('df_realestate.csv')

In [17]:
new_output = pd.read_csv('df_realestate.csv', keep_default_na=False, na_values=[""])
# new_output.head(20)
print(new_output.shape[0])
print(new_output.shape[1])




760
64
